<a href="https://colab.research.google.com/github/maria-natale/ProgettoFVAB/blob/main/Code/3blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!git clone https://github.com/maria-natale/ProgettoFVAB

Cloning into 'ProgettoFVAB'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 97 (delta 23), reused 51 (delta 8), pack-reused 0
Unpacking objects: 100% (97/97), done.


In [12]:
!python "/content/ProgettoFVAB/Code/create_datasetcsv.py"

100% 195/195 [06:30<00:00,  2.00s/it]
100% 53/53 [01:06<00:00,  1.25s/it]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bunch

  Created wheel for bunch: filename=bunch-1.0.1-cp37-none-any.whl size=7076 sha256=4e8654116eb443c276aa153ef29c18d9c7769f443e333c3551feb9bbb3f11932
  Stored in directory: /root/.cache/pip/wheels/56/0f/19/fbbf81e5764e6d8b74501c4357a88c14c94466ec777c03734c
Successfully built bunch


In [13]:
import os
import pandas as pd
import numpy as np

path_git = '/content/ProgettoFVAB'
filename = 'spagnolo_giapponese'
path_drive = '/content/drive/MyDrive/Casillo&Natale'
dataset_dir = 'dataset_Spagnolo_giapponese/datasetCSV'
LANGUAGES = {4:'Spagnolo',
  7: 'Giapponese'}

MAX_LINES = 360

Non serve

In [ ]:
from tqdm import tqdm 
from bunch import Bunch

def load_dataset():
  x_train = []
  y_train = []
  x_test = []
  y_test = []
  os.chdir(os.path.join(path_drive, dataset_dir, 'train_csv'))
  for file_name in tqdm(os.listdir()):
    data = []
    df = pd.read_csv(file_name)
    for i, row in df.iterrows() :
      if(i<MAX_LINES):
        new_row = [float (x) for x in row]
        for element in new_row :
          data.append(element)
      x_train.append (np.array(data))
      y_train.append(int(file_name.split('_')[0]))
  
  os.chdir(os.path.join(path_drive, dataset_dir, 'test_csv'))
  for file_name in tqdm(os.listdir()):
    data = []
    df = pd.read_csv(file_name)
    for i, row in df.iterrows() :
      if(i<MAX_LINES):
        new_row = [float (x) for x in row]
        for element in new_row :
          data.append(element)
      x_test.append (np.array(data))
      y_test.append(int(file_name.split('_')[0]))
  return x_train, y_train, x_test, y_test

In [ ]:
x_train, y_train, x_test, y_test = load_dataset()

100%|██████████| 53/53 [00:18<00:00,  2.90it/s]


**Eseguire da qua**

In [15]:
os.chdir(os.path.join(path_git, 'file_dataset', 'csv'))
x_train = pd.read_csv(filename+"_all_train.csv")
x_test = pd.read_csv(filename+"_all_test.csv")
y_train = pd.read_csv(filename+"targets_all_train.csv")
y_test = pd.read_csv(filename+"targets_all_test.csv")

In [18]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, LSTM
#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
#Reshape the data into 3-D array
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# Initialising the RNN
model = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[0], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and Dropout layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and and Dropout layer
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
# For Full connection layer we use dense
# As the output is 1D so we use unit=1
model.add(Dense(units = 1))

In [19]:

#compile and fit the model on 30 epochs
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(x_train, y_train, epochs = 30, batch_size = 50)

Epoch 1/30
1644/1644 [==============================] - 2270s 1s/step - loss: nan
Epoch 2/30
 211/1644 [==>...........................] - ETA: 34:01 - loss: nan

KeyboardInterrupt: ignored

# Nuova sezione

In [ ]:
#Convert x_test to a numpy array 
x_test = np.array(x_test)

#Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
from sklearn.matrix import confusion_matrix 

#check predicted values
predictions = model.predict(x_test) 
#Undo scaling
predictions = scaler.inverse_transform(predictions)

#Calculate RMSE score
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

#confusion matrix
confusion = confusion_matrix (y_test, predictions)
confusion